In [4]:
pip install codecarbon

You should consider upgrading via the '/Users/jules/Projects/green-hackathon/env/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [5]:
! codecarbon init

/Users/jules/Projects/green-hackathon/env/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(

Welcome to CodeCarbon, here is your experiment id:
73c54d65-6fc8-4a34-bef1-c56b3a73de67 (from ./.codecarbon.config)



In [8]:
## CNN 
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms

train_dir = 'data/train_images'
test_dir = 'data/test_images'

transform = transforms.Compose(
    [transforms.Grayscale(),
     transforms.ToTensor(),
     transforms.Normalize(mean=(0,),std=(1,))])

train_data = torchvision.datasets.ImageFolder(train_dir, transform=transform)
test_data = torchvision.datasets.ImageFolder(test_dir, transform=transform)

In [9]:
from torch.utils.data.sampler import SubsetRandomSampler

valid_size = 0.2
batch_size = 32
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

num_train = len(train_data)
indices_train = list(range(num_train))
np.random.shuffle(indices_train)
split_tv = int(np.floor(valid_size * num_train))
train_new_idx, valid_idx = indices_train[split_tv:],indices_train[:split_tv]

train_sampler = SubsetRandomSampler(train_new_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=train_sampler, num_workers=5)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=valid_sampler, num_workers=5)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=5)
classes = ('noface','face')

In [10]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 6 * 6, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 6 * 6)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model_3_layers_no_augmentation = Net().to(device)

In [12]:
def train_NN(n_epochs, optimizer, model, criterion):
    for epoch in range(1, n_epochs+1):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # print statistics after every epoch
        print(f'[{epoch}, {i + 1:5d}] loss: {running_loss / len(train_loader):.3f}')

    print('Finished Training')

In [13]:
print(len(train_loader))

2270


In [14]:
import torch.optim as optim
from codecarbon import EmissionsTracker

criterion = nn.CrossEntropyLoss()
optimizer_SGD = optim.SGD(model_3_layers_no_augmentation.parameters(), lr=0.001, momentum=0.9)
n_epochs = 20

tracker = EmissionsTracker()
tracker.start()
train_NN(n_epochs, optimizer_SGD, model_3_layers_no_augmentation, criterion)
emissions: float = tracker.stop()
print(emissions)



[codecarbon INFO @ 15:28:47] [setup] RAM Tracking...
[codecarbon INFO @ 15:28:47] [setup] GPU Tracking...
[codecarbon INFO @ 15:28:47] No GPU found.
[codecarbon INFO @ 15:28:47] [setup] CPU Tracking...
[codecarbon WARNING @ 15:28:47] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 15:28:47] CPU Model on constant consumption mode: Apple M1
[codecarbon INFO @ 15:28:47] >>> Tracker's metadata:
[codecarbon INFO @ 15:28:47]   Platform system: macOS-14.0-arm64-arm-64bit
[codecarbon INFO @ 15:28:47]   Python version: 3.9.6
[codecarbon INFO @ 15:28:47]   CodeCarbon version: 2.3.1
[codecarbon INFO @ 15:28:47]   Available RAM : 16.000 GB
[codecarbon INFO @ 15:28:47]   CPU count: 8
[codecarbon INFO @ 15:28:47]   CPU model: Apple M1
[codecarbon INFO @ 15:28:47]   GPU count: None
[codecarbon INFO @ 15:28:47]   GPU model: None
/Users/jules/Projects/green-hackathon/env/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports O

[1,  2270] loss: 0.595


/Users/jules/Projects/green-hackathon/env/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/jules/Projects/green-hackathon/env/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/jules/Projects/green-hackathon/env/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/jules/Projects/green-hackathon/env/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports

[2,  2270] loss: 0.236


/Users/jules/Projects/green-hackathon/env/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/jules/Projects/green-hackathon/env/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/jules/Projects/green-hackathon/env/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/jules/Projects/green-hackathon/env/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports

...

/Users/jules/Projects/green-hackathon/env/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/jules/Projects/green-hackathon/env/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/jules/Projects/green-hackathon/env/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/jules/Projects/green-hackathon/env/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports

[19,  2270] loss: 0.010


/Users/jules/Projects/green-hackathon/env/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/jules/Projects/green-hackathon/env/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/jules/Projects/green-hackathon/env/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/jules/Projects/green-hackathon/env/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports

[20,  2270] loss: 0.009
Finished Training
0.0001901408994799628


/Users/jules/Projects/green-hackathon/env/lib/python3.9/site-packages/codecarbon/output.py:168: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame.from_records([dict(data.values)])])
